In [2]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import re
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import ast
zomato=pd.read_csv("zomato.csv")
zomato.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [3]:
#unnecssary url,phone
zomato.drop(["url","phone"],axis=1,inplace=True)
zomato.head()

,address,name,online_order,book_table,rate,votes,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [4]:
zomato.isnull().sum()
zomato.dropna(inplace=True)

In [5]:
print(zomato.duplicated().sum())
zomato.drop_duplicates(inplace=True)

11


In [6]:
zomato["name"].describe()

count      23395
unique      3174
top       Onesta
freq          85
Name: name, dtype: object

In [7]:
zomato.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23395 entries, 0 to 51715
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   address                      23395 non-null  object
 1   name                         23395 non-null  object
 2   online_order                 23395 non-null  object
 3   book_table                   23395 non-null  object
 4   rate                         23395 non-null  object
 5   votes                        23395 non-null  int64 
 6   location                     23395 non-null  object
 7   rest_type                    23395 non-null  object
 8   dish_liked                   23395 non-null  object
 9   cuisines                     23395 non-null  object
 10  approx_cost(for two people)  23395 non-null  object
 11  reviews_list                 23395 non-null  object
 12  menu_item                    23395 non-null  object
 13  listed_in(type)              23395 n

In [8]:
zomato.columns

Index(['address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'location', 'rest_type', 'dish_liked', 'cuisines',
       'approx_cost(for two people)', 'reviews_list', 'menu_item',
       'listed_in(type)', 'listed_in(city)'],
      dtype='object')

In [9]:
zomato.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'type','listed_in(city)':'city','rate':'rating'},inplace=True)

In [10]:
zomato.cost

0          800
1          800
2          800
3          300
4          600
         ...  
51705    1,200
51707    2,000
51708    1,200
51711      800
51715    2,500
Name: cost, Length: 23395, dtype: object

In [11]:
zomato['cost'] = zomato['cost'].astype(str) #Changing the cost to string
zomato['cost'] = zomato['cost'].apply(lambda x: x.replace(',','')) #Using lambda function to replace ',' from cost
zomato['cost'] = zomato['cost'].astype(float) # Changing the cost to Float
zomato.cost

0         800.0
1         800.0
2         800.0
3         300.0
4         600.0
          ...  
51705    1200.0
51707    2000.0
51708    1200.0
51711     800.0
51715    2500.0
Name: cost, Length: 23395, dtype: float64

In [12]:
zomato.rating

0         4.1/5
1         4.1/5
2         3.8/5
3         3.7/5
4         3.8/5
          ...  
51705    3.8 /5
51707    3.9 /5
51708    2.8 /5
51711    2.5 /5
51715    4.3 /5
Name: rating, Length: 23395, dtype: object

In [13]:
#removing "/5" from rating as it is common in all
ind=zomato[(zomato.rating=="NEW") | (zomato.rating=="-")].index
zomato.drop(index=ind,inplace=True)
zomato["rating"]=zomato.rating.str.replace("/5","")

In [14]:
zomato["rating"]=zomato.rating.str.strip()
zomato["rating"]=zomato.rating.astype('float')
zomato.head()

,address,name,online_order,book_table,rating,votes,location,rest_type,dish_liked,cuisines,cost,reviews_list,menu_item,type,city
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1,775,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1,787,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800.0,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8,918,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800.0,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7,88,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300.0,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8,166,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600.0,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [15]:
zomato.name = zomato.name.apply(lambda x:x.title())
zomato.online_order.replace(('Yes','No'),(True, False),inplace=True)
zomato.book_table.replace(('Yes','No'),(True, False),inplace=True)
zomato.head()

,address,name,online_order,book_table,rating,votes,location,rest_type,dish_liked,cuisines,cost,reviews_list,menu_item,type,city
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,775,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,True,False,4.1,787,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800.0,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,True,False,3.8,918,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800.0,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,False,False,3.7,88,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300.0,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,False,False,3.8,166,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600.0,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [16]:
#restaurant are not unique
zomato["Mean Rating"]=0
from  collections import defaultdict
d={}
for ind,i in enumerate(zomato["name"]):d[i]=d.get(i,0)+zomato.iloc[ind].rating
for val in d:zomato["Mean Rating"][zomato.name==val]=d[val]/(len(zomato[zomato.name==val]))
zomato.head()

,address,name,online_order,book_table,rating,votes,location,rest_type,dish_liked,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,775,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari,4.118182
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,True,False,4.1,787,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800.0,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari,4.100000
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,True,False,3.8,918,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800.0,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari,3.800000
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,False,False,3.7,88,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300.0,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari,3.700000
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,False,False,3.8,166,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600.0,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari,3.800000


In [17]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (1,5))
zomato[['Mean Rating']] = scaler.fit_transform(zomato[['Mean Rating']]).round(2)
zomato.sample(10)

,address,name,online_order,book_table,rating,votes,location,rest_type,dish_liked,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
22528,"7th Main, NS Palya, 2nd Stage, BTM, Bangalore",Nanna Munna Paratha House,True,False,4.0,109,BTM,Quick Bites,"Paratha, Palak Paneer",North Indian,300.0,"[('Rated 5.0', 'RATED\n I ordered Sattu Parat...",[],Delivery,JP Nagar,3.84
10961,"115, First Floor, Opposite Raheja Arcade, Kora...",Rajdhani Thali Restaurant,True,True,4.0,792,Koramangala 7th Block,Casual Dining,"Rabri, Gatte Ki Subzi, Dhokla, Jalebi, Rajasth...",North Indian,900.0,"[('Rated 4.0', 'RATED\n Authentic rajasthani ...",[],Dine-out,BTM,3.78
15210,"Ramada Hotel, 11, Park Road, Shivajinagar, Ban...",Tiger Trail - Ramada Hotel,True,True,4.0,188,Shivajinagar,"Casual Dining, Bar","Biryani, Custard, Paratha, Poha, Breakfast Buf...","North Indian, Lucknowi",1300.0,"[('Rated 1.0', 'RATED\n Is this place really ...","['Tiger Trail Butter Chicken', 'Kakori Kebab N...",Delivery,Frazer Town,3.84
15619,"32, Annaswamy Mudaliar Road, Opposite Ulsoor L...",Urban Solace - CafãÂÃÂÃÂÃÂÃÂÃÂÃÂ...,True,False,3.9,462,Ulsoor,"Cafe, Casual Dining","Chocolate Shake, Wine, Burgers, Sangria, Parat...","Cafe, American, Continental, Steak",800.0,"[('Rated 4.0', ""RATED\n Like it's name a real...",[],Dine-out,Frazer Town,3.71
38273,"2, St. Patrick's Shopping Arcade, MG Road, Ban...",Stars 'N' Stripes,False,False,2.7,77,MG Road,Quick Bites,"Burgers, Biryani","North Indian, Chinese, Fast Food",500.0,"[('Rated 4.0', 'RATED\n good food and very go...",[],Delivery,Lavelle Road,2.16
40944,"Ground Floor, Vims Hospital Buidling, Opposite...",Bahar Cafe,True,True,4.0,914,Marathahalli,Casual Dining,"Chicken Biryani, Hyderabadi Biryani, Butter Na...","Andhra, Biryani, Chinese, Hyderabadi, North In...",750.0,"[('Rated 3.0', 'RATED\n We were lil lazy to g...","['Veg Biryani', 'Paneer Biryani', 'Mushroom Bi...",Delivery,Marathahalli,3.84
48174,"8, 2nd Floor, Allied Grande Plaza, Diagonally ...",Nolimmits Lounge And Club,True,True,4.1,2180,Brigade Road,Club,"Salads, Chicken Bbq Wings, Sandwiches, Pasta, ...","Chinese, Continental, North Indian",2000.0,"[('Rated 1.0', ""RATED\n Was really disappoint...",[],Dine-out,Residency Road,3.90
33990,"25, 1st Floor, 5th Cross, Koramangala 5th Bloc...",Chinita Real Mexican Food,False,False,4.5,739,Koramangala 5th Block,Casual Dining,"Tacos, Churros, Nachos, Salad, Enchiladas, Bur...",Mexican,1200.0,"[('Rated 4.0', ""RATED\n Been to this place co...",[],Dine-out,Koramangala 6th Block,4.47
9095,"178, 9th Main Road, Opposite Union Bank, Secto...",Moolehatti Donne Biryani Mane,True,False,3.6,218,HSR,Quick Bites,"Mutton Biryani, Chicken Kebab, Donne Biryani","Biryani, Chinese, North Indian, Kebab",400.0,"[('Rated 1.0', 'RATED\n I ordered veg-biriyan...",[],Delivery,BTM,3.32
20034,"11/4, Old Market Road, V V Puram, Basavanagudi...",The Chaat Shop,True,False,2.5,35,Basavanagudi,Quick Bites,"Badam Milk, Potato Twister, Pav Bhaji, Panipur...",Fast Food,200.0,[],"['Masala Puri', 'Panipuri', 'Shahi Tawa Pulao'...",Delivery,Jayanagar,1.90


In [18]:
import string
zomato["reviews_list"] = zomato["reviews_list"].str.lower()
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))
zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_punctuation(text))
zomato.head()

,address,name,online_order,book_table,rating,votes,location,rest_type,dish_liked,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,775,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800.0,rated 40 ratedn a beautiful place to dine int...,[],Buffet,Banashankari,3.99
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,True,False,4.1,787,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800.0,rated 40 ratedn had been here for dinner with...,[],Buffet,Banashankari,3.97
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,True,False,3.8,918,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800.0,rated 30 ratedn ambience is not that good eno...,[],Buffet,Banashankari,3.58
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,False,False,3.7,88,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300.0,rated 40 ratedn great food and proper karnata...,[],Buffet,Banashankari,3.45
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,False,False,3.8,166,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600.0,rated 40 ratedn very good restaurant in neigh...,[],Buffet,Banashankari,3.58


In [19]:
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_stopwords(text))
zomato.head()

,address,name,online_order,book_table,rating,votes,location,rest_type,dish_liked,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,775,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800.0,rated 40 ratedn beautiful place dine inthe int...,[],Buffet,Banashankari,3.99
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,True,False,4.1,787,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800.0,rated 40 ratedn dinner family turned good choo...,[],Buffet,Banashankari,3.97
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,True,False,3.8,918,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800.0,rated 30 ratedn ambience good enough pocket fr...,[],Buffet,Banashankari,3.58
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,False,False,3.7,88,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300.0,rated 40 ratedn great food proper karnataka st...,[],Buffet,Banashankari,3.45
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,False,False,3.8,166,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600.0,rated 40 ratedn good restaurant neighbourhood ...,[],Buffet,Banashankari,3.58


In [20]:
zomato.set_index('name', inplace=True)
indices = pd.Series(zomato.index)
# Adjust parameters to reduce the size of the TF-IDF matrix
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=5, max_features=5000, stop_words='english')
tfidf_matrix = tfidf.fit_transform(zomato['reviews_list'])

In [21]:
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

# Choose the number of components for TruncatedSVD =100
svd = TruncatedSVD(n_components=100)
tfidf_matrix_reduced = svd.fit_transform(tfidf_matrix)
# Compute cosine similarities using the reduced TF-IDF matrix
cosine_similarities = cosine_similarity(tfidf_matrix_reduced, dense_output=False)

In [63]:
def recommend(rest_name):
    restaurant_name=[]
    index=indices[indices==rest_name].index
    v=[[x,i] for i,x in enumerate(cosine_similarities[index[0]])]
    v.sort(reverse=True)
    v=v[1:101]
    for val,each in v:
        if list(zomato.index)[each]!=rest_name:
            restaurant_name.append(list(zomato.index)[each])
    df_new = pd.DataFrame(columns=['cuisines', 'Mean Rating', 'cost'])
    for each in restaurant_name:
        df_new=pd.concat([df_new,pd.DataFrame(zomato[['cuisines','Mean Rating', 'cost']][zomato.index == each].sample())],axis=0)
    df_new["name"]=restaurant_name
    # Drop the same named restaurants and sort only the top 10 by the highest rating
    #df_new = df_new.drop_duplicates(subset=['cuisines','Mean Rating', 'cost'])
    df_new=df_new.drop_duplicates(subset=['name'])
    df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(10)
    return df_new

In [64]:
df=recommend("Keventers")
df

,cuisines,Mean Rating,cost,name
Shakes Theory,"Beverages, Sandwich",4.48,300.0,Shakes Theory
The Thickshake Factory,"Beverages, Juices",4.08,350.0,The Thickshake Factory
Shake It Off,"Beverages, Fast Food",4.06,600.0,Shake It Off
Tempteys,Beverages,4.05,500.0,Tempteys
Makers Of Milkshakes,Beverages,4.02,300.0,Makers Of Milkshakes
The London Shakes,"Fast Food, Beverages",3.97,300.0,The London Shakes
Grapevine,"Cafe, Continental, Italian, Beverages, Mediter...",3.97,1200.0,Grapevine
Frozen Bottle,"Beverages, Desserts",3.92,400.0,Frozen Bottle
Drunken Monkey,"Beverages, Fast Food",3.88,400.0,Drunken Monkey
Cafe Orio,"Cafe, Fast Food",3.84,400.0,Cafe Orio


In [65]:
recommendations={}
cnt=0
for name in indices.unique():
    print(cnt)
    recommendations[name]=recommend(name)
    cnt+=1
recommendations

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

{'Jalsa':                                                                              cuisines  \
 Flechazo                                      Asian, Mediterranean, North Indian, BBQ   
 Ab'S - Absolute Barbecues                  European, Mediterranean, North Indian, BBQ   
 The Black Pearl                            North Indian, European, Mediterranean, BBQ   
 Barbeque Nation                     North Indian, European, Mediterranean, BBQ, Kebab   
 Jalsa Gold                                             North Indian, Mughlai, Italian   
 Buff Buffet Buff                      North Indian, Chinese, Continental, Thai, Salad   
 The Globe Grub                              Continental, North Indian, Asian, Italian   
 3B'S - Buddies, Bar & Barbecues            North Indian, Mediterranean, European, BBQ   
 Jw Kitchen - Jw Marriott Bengaluru                 North Indian, Continental, Italian   
 The 13Th Floor                                              Asian, North Indian, Thai   
 

In [66]:
pickle.dump(recommendations,open("recommendation.pkl","wb"))